In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns

from octopus_dash import data

figsize = (10, 6)

In [ ]:
with open("../donttrack/config.json") as tf:
    config = json.load(tf)

In [ ]:
d = data\
        .data_getter(**config)\
        .load_electric_data()

In [ ]:
df = d.enrich_data().get_dataframe()

In [ ]:
df.sample(10)

In [ ]:
monthly_use = df.groupby(["off_peak", "month"])["consumption"].sum().unstack().T

In [ ]:
import pandas as pd

In [ ]:
pd.concat([
    (monthly_use[False]/monthly_use.sum(axis=1))*0.3083,
    (monthly_use[True]/monthly_use.sum(axis=1))*0.0750
], axis = 1).sum(axis = 1)

In [ ]:
pd.concat([
    (monthly_use[False]/monthly_use.sum(axis=1))*0.3995,
    (monthly_use[True]/monthly_use.sum(axis=1))*0.12
], axis = 1).sum(axis = 1)

In [ ]:
def plot_by_month_and_off_peak(x, **kwargs):
    ax = df\
        .groupby(["off_peak", "month"])\
        [x]\
        .sum()\
        .unstack()\
        .T\
        .plot(
            kind = "bar",
            figsize = figsize,
            **kwargs
        )

    _ = ax.set_ylabel(x)
    
    return ax

In [ ]:
ax = plot_by_month_and_off_peak("consumption")
_ = ax.set_title("Consumption Split By Peak and Off-Peak")

In [ ]:
ax = plot_by_month_and_off_peak("cost")
_ = ax.set_title("Cost Split By Peak and Off-Peak")

In [ ]:
consumption = df\
    .groupby(["month", "off_peak"])\
    ["consumption"]\
    .mean()\
    .unstack()

total = consumption.sum(axis = 1)

total_norm = consumption / pd.concat([total, total], axis=1)

effective_rate = total_norm[False]*d.tariff.peak_rate + total_norm[True]*d.tariff.off_peak_rate

In [ ]:
ax = effective_rate.plot(kind="bar", figsize = figsize)
_ = ax.set_title("Effective Unit Rate")

In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))
ax = sns.heatmap(
    df.groupby(["hour", "month"])["cost"].mean().unstack().loc[:,3:]
)
_ = ax.set_title("Average energy use by hour and month")

In [ ]:
fig, ax = plt.subplots(figsize = (15, 6))
ax = sns.heatmap(
    df.groupby(["hour", "month"])["cost"].mean().unstack().loc[4:23.5,3:11]
)
_ = ax.set_title("Average energy use by hour and month")

In [ ]:
ax = df\
    .groupby(["month","hour"])\
    ["cost"]\
    .mean()\
    .unstack()\
    .loc[3:]\
    .T\
    .plot(figsize=(10,6))

_ = ax.set_title("Average energy use by hour and month")

## Estmating the effect of change in tarrif

In [ ]:
d.get_monthly_cost()

In [ ]:
d.tariff.off_peak_rate = 12.00/100
d.tariff.peak_rate = 39.95/100
d.standing_day_charge = 49.11/100

In [ ]:
d.get_monthly_cost()

In [ ]:
import pandas as pd

In [ ]:
df = df.assign(interval_start = lambda x: pd.to_datetime(x.interval_start, utc = True))

In [ ]:
df.set_index("interval_start", inplace=True)

In [ ]:
df.groupby(df.index.month)["off_grid"].sum().plot(kind="bar")

In [ ]:
df.groupby([df.index.month, df.off_peak])["cost"].sum().unstack().loc[3:].plot(kind="area")

In [ ]:
monthly_use = df.groupby(["month", "off_peak"])["consumption"].sum().unstack()
savings = monthly_use[False]*d.tariff.peak_rate-monthly_use[False]*d.tariff.off_peak_rate

peak_use = df\
    .assign(day = lambda x: x.interval_start.apply(lambda y: y.day))\
    .query("off_peak==False")\
    .groupby(["month", "day"])["consumption"].sum().unstack().T

battry_size = peak_use.mean() + 3*peak_use.std()

battery = pd.concat(
    [ savings.rename("savings"), battry_size.rename("kwh_per_day")]
,
        axis = 1
)

battery